In [1]:
import httpx
import polars as pl

## Data

### FLoRes-200

In [2]:
df_flores = pl.read_excel(
    "../data/processed/flores-200-language-database.xlsx", sheet_name="FLORES200P"
)

### Ethnologue

In this Jupyter notebook, we are downloading data from [Ethnologue.com](http://ethnologue.com/), a comprehensive reference work cataloging all of the world's known living languages. Ethnologue provides valuable information about languages, including their classification, population statistics, geographical distribution, and linguistic features. 

In [3]:
df_ethnologue = pl.read_csv("../data/raw/LanguageIndex.tab", separator="\t")

### World Bank Country API 

The [World Bank Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries) returns the current (as of 2024) [World Bank Country and Lending Groups](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups).

In [4]:
httpx.get("http://api.worldbank.org/v2/incomelevel?format=json").json()

[{'page': '1', 'pages': '1', 'per_page': '50', 'total': '7'},
 [{'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
  {'id': 'INX', 'iso2code': 'XY', 'value': 'Not classified'},
  {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'},
  {'id': 'LMC', 'iso2code': 'XN', 'value': 'Lower middle income'},
  {'id': 'LMY', 'iso2code': 'XO', 'value': 'Low & middle income'},
  {'id': 'MIC', 'iso2code': 'XP', 'value': 'Middle income'},
  {'id': 'UMC', 'iso2code': 'XT', 'value': 'Upper middle income'}]]

Let's retrieve all countries and their respective income levels.

In [5]:
df_countries = (
    pl.DataFrame(
        httpx.get(
            "http://api.worldbank.org/v2/country?format=json&per_page=1000"
        ).json()[
            1
        ]  # see https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries
    )
    .with_columns(
        pl.col("region").map_elements(lambda x: x["value"]).alias("region"),
        pl.col("adminregion").map_elements(lambda x: x["value"]).alias("adminregion"),
        pl.col("incomeLevel").map_elements(lambda x: x["value"]).alias("incomeLevel"),
        pl.col("lendingType").map_elements(lambda x: x["value"]).alias("lendingType"),
    )
    .filter(pl.col("incomeLevel") != "Aggregates")
    .filter(pl.col("incomeLevel") != "NA")
)

In [6]:
df_countries

id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
str,str,str,str,str,str,str,str,str,str
"""ABW""","""AW""","""Aruba""","""Latin America …","""""","""High income""","""Not classified…","""Oranjestad""","""-70.0167""","""12.5167"""
"""AFG""","""AF""","""Afghanistan""","""South Asia""","""South Asia""","""Low income""","""IDA""","""Kabul""","""69.1761""","""34.5228"""
"""AGO""","""AO""","""Angola""","""Sub-Saharan Af…","""Sub-Saharan Af…","""Lower middle i…","""IBRD""","""Luanda""","""13.242""","""-8.81155"""
"""ALB""","""AL""","""Albania""","""Europe & Centr…","""Europe & Centr…","""Upper middle i…","""IBRD""","""Tirane""","""19.8172""","""41.3317"""
"""AND""","""AD""","""Andorra""","""Europe & Centr…","""""","""High income""","""Not classified…","""Andorra la Vel…","""1.5218""","""42.5075"""
…,…,…,…,…,…,…,…,…,…
"""XKX""","""XK""","""Kosovo""","""Europe & Centr…","""Europe & Centr…","""Upper middle i…","""IDA""","""Pristina""","""20.926""","""42.565"""
"""YEM""","""YE""","""Yemen, Rep.""","""Middle East & …","""Middle East & …","""Low income""","""IDA""","""Sana'a""","""44.2075""","""15.352"""
"""ZAF""","""ZA""","""South Africa""","""Sub-Saharan Af…","""Sub-Saharan Af…","""Upper middle i…","""IBRD""","""Pretoria""","""28.1871""","""-25.746"""


## Merging

First, merging languages with countries, 

In [7]:
df_languages = df_ethnologue.join(
    df_countries, left_on="CountryID", right_on="iso2Code", how="outer"
)

Secondly, merging languages/countries with FLoRes-200, 

In [8]:
df_flores.with_columns(pl.col("code").map_elements(lambda x: x[:3]).alias("code")).join(
    df_languages, left_on="code", right_on="LangID", how="left"
)

language,code,countries,num_speakers,CountryID,NameType,Name,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Acehnese (Arab…","""ace""",null,null,"""ID""","""D""","""Banda Aceh""","""IDN""","""ID""","""Indonesia""","""East Asia & Pa…","""East Asia & Pa…","""Upper middle i…","""IBRD""","""Jakarta""","""106.83""","""-6.19752"""
"""Acehnese (Arab…","""ace""",null,null,"""ID""","""D""","""Baruh""","""IDN""","""ID""","""Indonesia""","""East Asia & Pa…","""East Asia & Pa…","""Upper middle i…","""IBRD""","""Jakarta""","""106.83""","""-6.19752"""
"""Acehnese (Arab…","""ace""",null,null,"""ID""","""D""","""Bueng""","""IDN""","""ID""","""Indonesia""","""East Asia & Pa…","""East Asia & Pa…","""Upper middle i…","""IBRD""","""Jakarta""","""106.83""","""-6.19752"""
"""Acehnese (Arab…","""ace""",null,null,"""ID""","""D""","""Daja""","""IDN""","""ID""","""Indonesia""","""East Asia & Pa…","""East Asia & Pa…","""Upper middle i…","""IBRD""","""Jakarta""","""106.83""","""-6.19752"""
"""Acehnese (Arab…","""ace""",null,null,"""ID""","""D""","""Pase""","""IDN""","""ID""","""Indonesia""","""East Asia & Pa…","""East Asia & Pa…","""Upper middle i…","""IBRD""","""Jakarta""","""106.83""","""-6.19752"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zulu""","""zul""",null,null,"""ZA""","""D""","""Transvaal Zulu…","""ZAF""","""ZA""","""South Africa""","""Sub-Saharan Af…","""Sub-Saharan Af…","""Upper middle i…","""IBRD""","""Pretoria""","""28.1871""","""-25.746"""
"""Zulu""","""zul""",null,null,"""ZA""","""L""","""Zulu""","""ZAF""","""ZA""","""South Africa""","""Sub-Saharan Af…","""Sub-Saharan Af…","""Upper middle i…","""IBRD""","""Pretoria""","""28.1871""","""-25.746"""
"""Zulu""","""zul""",null,null,"""ZA""","""LA""","""Isizulu""","""ZAF""","""ZA""","""South Africa""","""Sub-Saharan Af…","""Sub-Saharan Af…","""Upper middle i…","""IBRD""","""Pretoria""","""28.1871""","""-25.746"""


In [9]:
df_flores.with_columns(pl.col("code").map_elements(lambda x: x[:3]).alias("code")).join(
    df_languages, left_on="code", right_on="LangID", how="left"
).write_csv("../data/processed/language_country_income_level.csv")